### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-10-02


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-10-02
Yesterday: 2025-10-01


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}','price':'{:.2f}',
    'beta':'{:,.2f}','change':'{:.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}','volume':'{:,.0f}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-10-01'
ORDER BY a.name



(18, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-10-01' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df.style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
AIMIRT,2025-10-01,2025-10-01,10.30,10.40,"1,347,644",10.60,10.20,B,SET,0.10,0.97%
CPF,2025-09-30,2025-10-01,22.60,22.30,"240,514,224",22.60,22.00,B,SET50,-0.30,-1.33%
GVREIT,2025-09-26,2025-10-01,6.50,6.45,"280,920",6.50,6.45,B,SET,-0.05,-0.77%
IVL,2025-09-26,2025-10-01,22.40,22.00,"133,699,400",23.10,22.00,B,SET50,-0.40,-1.79%
JMART,2025-09-26,2025-10-01,9.75,8.95,"103,744,280",10.10,8.95,B,SET50,-0.80,-8.21%
JMT,2025-09-26,2025-10-01,12.50,11.30,"70,600,864",12.80,11.30,B,SET50,-1.20,-9.60%
NER,2025-09-29,2025-10-01,4.30,4.22,"78,901,536",4.30,4.18,B,SET,-0.08,-1.86%
ORI,2025-09-26,2025-10-01,2.82,2.56,"57,616,136",2.88,2.56,B,SET100,-0.26,-9.22%
RCL,2025-09-26,2025-10-01,27.00,26.00,"13,640,968",27.25,26.00,B,SET100,-1.00,-3.70%
TFFIF,2025-09-26,2025-10-01,6.15,6.20,"18,714,904",6.20,6.10,B,SET,0.05,0.81%


In [20]:
df.shape

(18, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
AIMIRT,2025-10-01,2025-10-01,10.30,10.40,"1,347,644",10.60,10.20,0.97%,B
CPF,2025-09-30,2025-10-01,22.60,22.30,"240,514,224",22.60,22.00,-1.33%,B
GVREIT,2025-09-26,2025-10-01,6.50,6.45,"280,920",6.50,6.45,-0.77%,B
IVL,2025-09-26,2025-10-01,22.40,22.00,"133,699,400",23.10,22.00,-1.79%,B
JMART,2025-09-26,2025-10-01,9.75,8.95,"103,744,280",10.10,8.95,-8.21%,B
JMT,2025-09-26,2025-10-01,12.50,11.30,"70,600,864",12.80,11.30,-9.60%,B
NER,2025-09-29,2025-10-01,4.30,4.22,"78,901,536",4.30,4.18,-1.86%,B
ORI,2025-09-26,2025-10-01,2.82,2.56,"57,616,136",2.88,2.56,-9.22%,B
RCL,2025-09-26,2025-10-01,27.00,26.00,"13,640,968",27.25,26.00,-3.70%,B
TFFIF,2025-09-26,2025-10-01,6.15,6.20,"18,714,904",6.20,6.10,0.81%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(18, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(18, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu].style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
AIMIRT,2025-10-01,2025-10-01,10.30,10.40,"1,347,644",10.60,10.20,0.97%,B,10.40,0.00,"54,538",2025-10-02
CPF,2025-09-30,2025-10-01,22.60,22.30,"240,514,224",22.60,22.00,-1.33%,B,22.30,0.00,"13,262,300",2025-10-02
GVREIT,2025-09-26,2025-10-01,6.50,6.45,"280,920",6.50,6.45,-0.77%,B,6.40,-0.05,"819,978",2025-10-02
IVL,2025-09-26,2025-10-01,22.40,22.00,"133,699,400",23.10,22.00,-1.79%,B,22.50,0.50,"10,107,191",2025-10-02
JMART,2025-09-26,2025-10-01,9.75,8.95,"103,744,280",10.10,8.95,-8.21%,B,9.15,0.20,"7,844,695",2025-10-02
JMT,2025-09-26,2025-10-01,12.50,11.30,"70,600,864",12.80,11.30,-9.60%,B,11.50,0.20,"8,118,053",2025-10-02
NER,2025-09-29,2025-10-01,4.30,4.22,"78,901,536",4.30,4.18,-1.86%,B,4.26,0.04,"5,335,136",2025-10-02
ORI,2025-09-26,2025-10-01,2.82,2.56,"57,616,136",2.88,2.56,-9.22%,B,2.64,0.08,"2,813,996",2025-10-02
RCL,2025-09-26,2025-10-01,27.00,26.00,"13,640,968",27.25,26.00,-3.70%,B,26.25,0.25,"2,463,476",2025-10-02
TFFIF,2025-09-26,2025-10-01,6.15,6.20,"18,714,904",6.20,6.10,0.81%,B,6.15,-0.05,"2,075,341",2025-10-02


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-10-01 2025-10-02  10.30  10.40   0.97%   1,347,644  10.60  10.20 B Update
CPF      2025-09-30 2025-10-02  22.60  22.30  -1.33% 240,514,224  22.60  22.00 B Update
DIF      2025-10-02 2025-10-02   9.10   9.05  -0.55%  10,276,338   9.20   9.05 S Insert
GVREIT   2025-09-26 2025-10-02   6.50   6.40  -1.54%     280,920   6.50   6.40 B Update
IVL      2025-10-02 2025-10-02  22.00  22.50   2.27% 133,699,400  23.10  22.00 B Insert
JMART    2025-10-02 2025-10-02   8.95   9.15   2.23% 103,744,280  10.10   8.95 B Insert
JMT      2025-10-02 2025-10-02  11.30  11.50   1.77%  70,600,864  12.80  11.30 B Insert
MCS      2025-10-02 2025-10-02   8.25   8.30   0.61%   4,657,152   8.45   8.25 S Insert
NER      2025-10-02 2025-10-02   4.22   4.26   0.95%  78,901,536   4.30   4.18 B Insert
ORI      2025-10-02 2025-10-02  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '{}' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name
""".format(today)
print(sql)


    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '2025-10-02' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(18, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
AIMIRT,2025-10-01,2025-10-02,10.30,10.40,"2,695,288",10.60,10.20,B,SET,0.10,0.97%
CPF,2025-09-30,2025-10-02,22.60,22.30,"481,028,448",22.60,22.00,B,SET50,-0.30,-1.33%
GVREIT,2025-09-26,2025-10-02,6.50,6.40,"561,840",6.50,6.40,B,SET,-0.10,-1.54%
IVL,2025-10-02,2025-10-02,22.00,22.50,"133,699,400",23.10,22.00,B,SET50,0.50,2.27%
JMART,2025-10-02,2025-10-02,8.95,9.15,"103,744,280",10.10,8.95,B,SET50,0.20,2.23%
JMT,2025-10-02,2025-10-02,11.30,11.50,"70,600,864",12.80,11.30,B,SET50,0.20,1.77%
NER,2025-10-02,2025-10-02,4.22,4.26,"78,901,536",4.30,4.18,B,SET,0.04,0.95%
ORI,2025-10-02,2025-10-02,2.56,2.64,"57,616,136",2.88,2.56,B,SET100,0.08,3.13%
RCL,2025-10-02,2025-10-02,26.00,26.25,"13,640,968",27.25,26.00,B,SET100,0.25,0.96%
TFFIF,2025-10-02,2025-10-02,6.20,6.15,"18,714,904",6.20,6.10,B,SET,-0.05,-0.81%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-02 20:34:00
